# Top 100 Airports Data Analysis with SQL

We have a dataset with Top 100 airports along with two different tables, GDP category of country, and number of international arrivals into the country.

## Connecting to Database
---

In [2]:
import pandas as pd
import pyodbc
import mysql.connector as connector
import numpy as np
# Establish a connection to the database
conn_str = 'DRIVER={SQL Server};SERVER=server_name;DATABASE=database_name;UID=username;PWD=password'

conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=DESKTOP-CPIGF08;'
                     'Database=airports;'
                     'Trusted_Connection=yes;')

print(conn)

# Create a cursor to execute queries
cursor = conn.cursor()

In [3]:
def run_query(query):

    # Execute a sample query
    cursor.execute(query)

    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(np.array(rows), columns=column_names)

    return df


## Understanding the Data
---
First, we start with looking at how the data is split and what factors are involved.

In [61]:
run_query("SELECT TOP 5 * FROM airports ")

,Rank,Airport,Code,Location,Country,Passengers
0,11,Amsterdam Airport Schiphol,AMS,Amsterdam,Netherlands,68515425
1,78,Stockholm Arlanda Airport,ARN,Stockholm,Sweden,26642034
2,1,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,103902992
3,95,Abu Dhabi International Airport,AUH,Abu Dhabi,United Arab Emirates,23760561
4,80,Antalya Airport,AYT,Antalya,Turkey,25931659


The facts table comprises **6 columns** with intuitive names:
>- **Rank** - Rank of each airport in terms of Passengers count.
>- **Airport** — The name of the airport.
>- **Code** — Code of the airport.
>- **Location** — City of the airport.
>- **Country** — Country of the airport.
>- **Passengers** — Number of passengers the airport has handled in a year.

## Exploring the Dataset
---
We can start by looking at which airport is ranked at 1 and which is ranked 100.

In [69]:
run_query("SELECT * FROM airports WHERE Passengers = (SELECT MAX(Passengers) FROM airports)")

,Rank,Airport,Code,Location,Country,Passengers
0,1,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,103902992


In [70]:
run_query("SELECT * FROM airports WHERE Passengers = (SELECT MIN(Passengers) FROM airports)")

,Rank,Airport,Code,Location,Country,Passengers
0,100,Wuhan Tianhe International Airport,WUH,Wuhan,China,23129400


This does not tell us much, let us instead look at the top 10 airports and their locations.

In [30]:
run_query("SELECT TOP 10 Airport,Passengers, Location FROM airports ORDER BY Passengers DESC")


,Airport,Passengers,Location
0,Hartsfield–Jackson Atlanta International Airport,103902992,Atlanta
1,Beijing Capital International Airport,95786442,Beijing
2,Dubai International Airport,88242099,Dubai
3,Los Angeles International Airport,84557968,Los Angeles
4,O'Hare International Airport,79828183,Chicago
5,Heathrow Airport,78014598,London
6,Haneda Airport,76476251,Tokyo
7,Hong Kong International Airport,72665078,Hong Kong
8,Shanghai Pudong International Airport,70001237,Shanghai
9,Charles de Gaulle International Airport,69471442,Paris


In [33]:
run_query("SELECT TOP 10 Country,COUNT(Country) AS Airports FROM airports GROUP BY Country ORDER BY 2 DESC")

,Country,Airports
0,United States,25
1,China,16
2,Japan,4
3,United Kingdom,4
4,Australia,3
5,Germany,3
6,India,3
7,South Korea,3
8,Spain,3
9,Turkey,3


>- United States has the most airports in Top 100. <br>
>- United States and China have combined 41% of top 100 airports. <br>


US has the most airports in Top 100, followed by China. Countries like Japan, UK, Australia have very low numbers compared to US and China. This shows which country has most air traffic compared to the rest. Let us see now how they fare against their respective GDP. 

### A. Country Analysis with GDP
---

In [54]:
run_query("SELECT TOP 10 airports.Country, CAST((CAST(SUM(airports.Passengers) AS decimal) / CAST(total_passengers.TotalPassengers AS decimal))*100 AS DECIMAL(3,1)) AS ProportionOfTotalPassengers, category.IncomeGroup FROM airports JOIN category ON category.TableName = airports.Country CROSS JOIN (SELECT SUM(CAST(Passengers AS bigint)) AS TotalPassengers FROM airports) AS total_passengers GROUP BY airports.Country, category.IncomeGroup, total_passengers.TotalPassengers ORDER BY ProportionOfTotalPassengers DESC ")

,Country,ProportionOfTotalPassengers,IncomeGroup
0,United States,27.4,High income
1,China,15.7,Upper middle income
2,United Kingdom,4.1,High income
3,Japan,3.9,High income
4,India,3.2,Lower middle income
5,Germany,3.1,High income
6,Spain,3.0,High income
7,United Arab Emirates,2.6,High income
8,Australia,2.3,High income
9,Thailand,2.3,Upper middle income


>- US and China alone make up more than 43% of all air traffic in the world. <br>
>- India, Thailand and China are the upcoming economies, especially India. <br>
>- India being a lower middle income economy is ranked 4th and has 3.2% of all air traffic comapred to many high income countires. <br>


GDP is termed as Income Group in the above table. From this it can be deduced that India, Thailand and China are the upcoming economies, especially India. Being in a lower income group means people don't earn a lot to be travelling frequently, and that it is either an upcoming economic hub, or a tourist destination, which is true for Thailand. The fact that India being a lower middle income economy is ranked 4th and has 3.2% of all air traffic comapred to high income countires or tourist destinations like Thailand, raises the question that is it busy due to international traffic or is it domestic travel that countributes to the major portion of the air traffic. 

### B. Country Analysis with International Arrival and its GDP
---
We now look at the International Arrivals from 2019 since the data is limited to 2020, and 2020 was affected by COVID-19 which will flaw our results, therefore the only viable option is 2019.

In [72]:
run_query("SELECT TOP 10 int_arrival.column64 AS InternationalArrivals2019 ,category.IncomeGroup, category.TableName AS Country  FROM category JOIN int_arrival ON category.TableName= int_arrival.column1 WHERE int_arrival.column65 IS NOT NULL ORDER BY 1 DESC")

,InternationalArrivals2019,IncomeGroup,Country
0,217876992.0,High income,France
1,165478000.0,High income,United States
2,162538000.0,Upper middle income,China
3,126170000.0,High income,Spain
4,97406000.0,Upper middle income,Mexico
5,95399000.0,High income,Italy
6,61397000.0,High income,Hungary
7,60021000.0,High income,Croatia
8,55913000.0,High income,"Hong Kong SAR, China"
9,40857000.0,High income,United Kingdom


>- France leads the world with its most international passengers, possibly because it is a tourist destination and a high income country where residents travel frequently. <br>
>- China and Mexico are the only countries outside high income bracket that show up in top 10 for most international passengers. <br>

As discussed above, India does not show up in top 10 for international arrivals, therefore the major portion of passengers are domestic travellers. We can now further look into how what portion of travellers do make the total passengers.

In [76]:
run_query("SELECT TOP 10 int_arrival.column64 as InternationalArrivals,SUM(airports.Passengers) AS TotalPassengers,ROUND((int_arrival.column64/SUM(airports.Passengers))*100,1) AS Proportion , category.IncomeGroup ,airports.Country FROM airports JOIN int_arrival ON int_arrival.column1 = airports.Country JOIN category ON category.TableName = airports.Country WHERE int_arrival.column64 IS NOT NULL GROUP BY int_arrival.column1,category.IncomeGroup, airports.Country,int_arrival.column64 ORDER BY 3 DESC ")


,InternationalArrivals,TotalPassengers,Proportion,IncomeGroup,Country
0,217876992.0,69471442,313.6,High income,France
1,95399000.0,40968756,232.9,High income,Italy
2,97406000.0,68333927,142.5,Upper middle income,Mexico
3,31884000.0,24392805,130.7,High income,Austria
4,33093000.0,29177833,113.4,High income,Denmark
5,126170000.0,128619556,98.1,High income,Spain
6,17283000.0,26669755,64.8,High income,Portugal
7,18009000.0,35900000,50.2,Lower middle income,Vietnam
8,32430000.0,71220818,45.5,High income,Canada
9,26101000.0,58558440,44.6,Upper middle income,Malaysia


The proportion cannot be more than 100 and 5 countries have more than 100% proportion. WIth this it can deduced that the data for international travellers and total passengers is not from the same year. The data for total passengers is recent, possibly from 2022, whereas we are limted by the international passsengers data which is updated up-to 2020. Instead of comparing total passengers to international, we can rathar look into internationals just by itself and see how COVID affected international travel from 2019 to 2020, the start of the pandemic.

In [83]:
run_query("SELECT TOP 10 int_arrival.column64 AS InternationalArrivals2019,int_arrival.column65 AS InternationalArrivals2020 ,ROUND((int_arrival.column65/int_arrival.column64)*100,2) as RateofChange,category.IncomeGroup, category.TableName AS Country  FROM category JOIN int_arrival ON category.TableName= int_arrival.column1 WHERE int_arrival.column65 IS NOT NULL ORDER BY 3")

,InternationalArrivals2019,InternationalArrivals2020,RateofChange,IncomeGroup,Country
0,19200.0,900.0,4.69,Upper middle income,American Samoa
1,55913000.0,3569000.0,6.38,High income,"Hong Kong SAR, China"
2,805000.0,51400.0,6.39,High income,Bermuda
3,316000.0,29800.0,9.43,Lower middle income,Bhutan
4,637000.0,66900.0,10.5,Lower middle income,Mongolia
5,1651000.0,187100.0,11.33,Upper middle income,Namibia
6,81000.0,10400.0,12.84,Low income,Chad
7,31881000.0,4115799.75,12.91,High income,Japan
8,181000.0,23900.0,13.2,Lower middle income,Samoa
9,688000.0,93000.0,13.52,Lower middle income,West Bank and Gaza


The countries that were most affected by the impact of COVID-19 are shown above, with a number of mixed economies. This shows that COVID-19 affected countries worldwide and it locked everything down. It wasn't just a particular economic set of countries being afftected which has been seen in previous queries. 

In [84]:
cursor.close()
conn.close()

## Conclusion
---

Throughout this project, we used the Top 100 Airports data to identify the how the countries around the world fare in terms of air traffic. Our analysis is based on qualitative and quantitative indicators like total passengers, international arrivals, and GDP.

>- United States and China have the most number of airports in top 100. <br>
>- Mixed number of econmoies were presented in the top 100 airports and not just high income countries. <br>
>- In terms of international travel, most high income and upper middle income economies lead the charge.  <br>
>- COVID-19 affected all economies and not certain groups, it slowed down travel everywhere.
  
## Limitation
---
  
- The data is only limited to 2020 for International Arrivals which limits further analysis to see which countries recovered from COVID the best and which were affected very badly. At the current stage, only the initial affect can be measured, which is not very ideal.
- The data on international arrivals has been updated until 2020; however, due to that we cannot compare how many international arrivals were there with the total passengers to give us a hollistic view.